In [2]:
import numpy as np
import pandas as pd
from itertools import islice
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

In [3]:
%run ../src/acc_df.py

In [4]:
Ac = Acc_data('../data/cleaned/coll_coord.csv')

In [5]:
acc = Ac.get_all_coll() #['']

#### Time series by week

In [7]:
acc_04_18 = acc[(acc['year'] != 2019) & (acc['year'] != 2003 )]
acc_w = acc_04_18[['year','week','OBJECTID']].groupby(['year','week']).count()
week_numbers = acc_w.reset_index()
week_numbers.rename({'OBJECTID': 'acc_nr'}, axis=1, inplace=True)
week_numbers.head(2)

,year,week,acc_nr
0,2004,1,142
1,2004,2,263


### other data / dummies

In [28]:
week_d = pd.get_dummies(week_numbers.week).join(week_numbers)
len(week_d)

786

In [30]:
# X=acc_ds[['year','month','week','day','weekday']]
X = week_d.drop(['acc_nr'], axis=1)
y=week_d['acc_nr']

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [32]:
model = GradientBoostingRegressor(learning_rate=0.001, 
                                  n_estimators=5000, #boosting stages to perform
                                  max_depth=3)
model.fit(X_train, y_train)

y_hat = model.predict(X_test)

In [33]:
model.feature_importances_

array([3.18495565e-03, 7.27503285e-03, 9.54573797e-03, 0.00000000e+00,
       2.56351887e-03, 0.00000000e+00, 0.00000000e+00, 3.33475109e-04,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 6.47718464e-05,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 2.89900034e-05, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 4.66604338e-04, 3.57282776e-03, 0.00000000e+00,
       0.00000000e+00, 9.62628770e-04, 7.78900319e-04, 0.00000000e+00,
       4.34855538e-04, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 4.41574526e-04, 0.00000000e+00, 7.98631422e-04,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       6.36187368e-05, 0.00000000e+00, 4.67549092e-03, 2.53251601e-02,
       0.00000000e+00, 0.00000000e+00, 4.95231014e-03, 0.00000000e+00,
       1.93595479e-04, 7.94129946e-03, 6.01928969e-03, 1.10296807e-02,
       4.54345534e-02, 5.59139789e-01, 3.04772707e-01])

In [34]:
model.score(X_train,y_train)

0.6510414682873786

In [35]:
model.score(X_test,y_test)

0.4093990931375383

In [51]:
model = GradientBoostingRegressor(learning_rate=0.001, 
                                  n_estimators=5000, #boosting stages to perform
                                  max_depth=3)
model.fit(X_train, y_train)

y_hat = model.predict(X_test)

In [52]:
model.score(X_train,y_train)

0.043033278207900705

In [53]:
model.score(X_test,y_test)

0.03538001428563897

In [67]:
if True:
    grid = {
        'learning_rate': [0.01,0.02,0.003, 0.004, 0.0005, 0.00001],
        'max_depth': [3,4,5,6,7,8,9,10],
        'min_samples_leaf': [1,2,3,4,5],
        'max_features': ['sqrt'],
        'n_estimators': [3000],
        'random_state': [0]
    }
else:  # TEST
    grid = {
    'learning_rate': [1],
    'max_depth': [2], 
    'min_samples_leaf': [2],
#     'max_features': ['sqrt', None],
    'n_estimators': [2],
    'random_state': [0]
}
    

from sklearn.model_selection import GridSearchCV
# confusion_score = make_scorer(confusion_rmse, greater_is_better=False)
gbc_grid_cv = GridSearchCV(
    GradientBoostingRegressor(), 
    grid,
    cv=4,  # number of folds
    return_train_score=True,
    verbose=1, 
    n_jobs=-1)
gbc_grid_cv.fit(X_train, y_train)
# Collapse

Fitting 4 folds for each of 240 candidates, totalling 960 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   26.5s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 960 out of 960 | elapsed:  2.8min finished
/Users/franzmornau/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=4, error_score='raise-deprecating',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_sampl...=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'learning_rate': [0.01, 0.02, 0.003, 0.004, 0.0005, 1e-05], 'max_depth': [3, 4, 5, 6, 7, 8, 9, 10], 'min_samples_leaf': [1, 2, 3, 4, 5], 'max_features': ['sqrt'], 'n_estimators': [3000], 'random_state': [0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=1)

In [68]:
gbc_grid_cv.best_estimator_

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=3,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=5, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=3000,
             n_iter_no_change=None, presort='auto', random_state=0,
             subsample=1.0, tol=0.0001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [69]:
gbc_grid_cv.best_score_

0.45830240211421214

In [70]:
pd.DataFrame(gbc_grid_cv.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_max_features,param_min_samples_leaf,param_n_estimators,param_random_state,...,split3_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,mean_train_score,std_train_score
0,1.012640,0.017156,0.008405,0.000312,0.01,3,sqrt,1,3000,0,...,0.380224,0.357935,0.051228,123,0.821083,0.855255,0.843811,0.819306,0.834863,0.015230
1,1.021743,0.017675,0.008768,0.000749,0.01,3,sqrt,2,3000,0,...,0.408606,0.392573,0.055240,95,0.746093,0.780389,0.782992,0.740852,0.762581,0.019220
2,1.008254,0.004139,0.008238,0.000072,0.01,3,sqrt,3,3000,0,...,0.472725,0.419739,0.041077,64,0.691227,0.732023,0.729896,0.681588,0.708683,0.022547
3,0.939851,0.010513,0.008745,0.000474,0.01,3,sqrt,4,3000,0,...,0.503251,0.445735,0.048285,12,0.650824,0.698906,0.700958,0.645484,0.674043,0.025968
4,0.895469,0.028653,0.008036,0.000234,0.01,3,sqrt,5,3000,0,...,0.514108,0.458302,0.058574,1,0.630140,0.686435,0.674557,0.610111,0.650311,0.031288
5,1.287092,0.010888,0.010691,0.000524,0.01,4,sqrt,1,3000,0,...,0.277624,0.284349,0.055814,166,0.913180,0.926920,0.917261,0.913584,0.917736,0.005535
6,1.197949,0.015947,0.010743,0.000148,0.01,4,sqrt,2,3000,0,...,0.359178,0.355965,0.069884,127,0.798267,0.819159,0.829165,0.789436,0.809007,0.015873
7,1.131016,0.013952,0.010290,0.000404,0.01,4,sqrt,3,3000,0,...,0.457095,0.408101,0.043571,78,0.724915,0.759405,0.756850,0.712996,0.738541,0.020054
8,1.064749,0.026131,0.009797,0.000189,0.01,4,sqrt,4,3000,0,...,0.505756,0.444430,0.054510,18,0.679517,0.720962,0.722236,0.670232,0.698237,0.023596
9,0.992711,0.014819,0.010213,0.001101,0.01,4,sqrt,5,3000,0,...,0.519273,0.455444,0.063926,2,0.654379,0.707697,0.693553,0.633824,0.672363,0.029606
